In [135]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [136]:
import pandas as pd
from pathlib import Path

In [137]:
file_dir = Path("/Users/admin/AMR/data/raw/data")
output_dir = Path("/Users/admin/AMR/data/cleaned")

In [151]:
import pandas as pd
from pathlib import Path

# Set your directories
ecoli_dir = Path("/Users/admin/AMR/data/raw/ecoli")  # <-- your new 8 CSVs folder
output_dir = Path("/Users/admin/AMR/data/cleaned")

print("="*70)
print("PROCESSING NEW E-COLI REGIONAL DATA")
print("="*70)

dataframes = []

print("\n--- E-COLI FILES ---")
for csv_file in ecoli_dir.glob("*.csv"):
    print(f"Reading: {csv_file.name}")
    df = pd.read_csv(csv_file)
    print(f"  Rows: {len(df)}")

    # --- Robust resistance column fix ---
    # Some rows have % in metric_value, others in metric value
    df['resistance_pct'] = pd.to_numeric(df['metric_value'], errors='coerce')


    dataframes.append(df)

# Combine all 8
print("\n" + "="*70)
combined = pd.concat(dataframes, ignore_index=True)
print(f"Total rows after combining: {len(combined)}")

# Convert date
combined['date'] = pd.to_datetime(combined['date'], dayfirst=True, errors='coerce')

# OPTIONAL: filter July & August 2025 (only if needed)
print("\n--- FILTERING OUT JULY & AUGUST 2025 ---")
print(f"Rows before filtering: {len(combined)}")

combined = combined[
    ~((combined['date'].dt.year == 2025) & (combined['date'].dt.month.isin([7, 8])))
]

print(f"Rows after filtering: {len(combined)}")

# Keep only clean essential columns
final_ecoli = combined[['geography', 'stratum', 'date', 'resistance_pct', 'topic']].copy()

# Drop rows missing true resistance or date
final_ecoli = final_ecoli.dropna(subset=['date', 'resistance_pct'])

# Sort
final_ecoli = final_ecoli.sort_values(['geography', 'stratum', 'date'])

# Save
output_path = output_dir / "ecoli_combined_cleaned.csv"
final_ecoli.to_csv(output_path, index=False)

print(f"\n✅ SAVED TO: {output_path}")
print(f"Total clean rows: {len(final_ecoli)}")

print("\nDate range:")
print("  Earliest:", final_ecoli['date'].min())
print("  Latest:", final_ecoli['date'].max())

print("\nRows per region:")
print(final_ecoli['geography'].value_counts())

print("\nFirst 5 rows:")
print(final_ecoli.head())



PROCESSING NEW E-COLI REGIONAL DATA

--- E-COLI FILES ---
Reading: ukhsa-chart-download (18).csv
  Rows: 413
Reading: ukhsa-chart-download (18) copy 8.csv
  Rows: 413
Reading: ukhsa-chart-download (18) copy.csv
  Rows: 413
Reading: ukhsa-chart-download (18) copy 4.csv
  Rows: 413
Reading: ukhsa-chart-download (18) copy 5.csv
  Rows: 413
Reading: ukhsa-chart-download (18) copy 7.csv
  Rows: 413
Reading: ukhsa-chart-download (18) copy 6.csv
  Rows: 413
Reading: ukhsa-chart-download (18) copy 2.csv
  Rows: 413
Reading: ukhsa-chart-download (18) copy 3.csv
  Rows: 413

Total rows after combining: 3717

--- FILTERING OUT JULY & AUGUST 2025 ---
Rows before filtering: 3717
Rows after filtering: 3717

✅ SAVED TO: /Users/admin/AMR/data/cleaned/ecoli_combined_cleaned.csv
Total clean rows: 3717

Date range:
  Earliest: 2020-01-10 00:00:00
  Latest: 2025-01-08 00:00:00

Rows per region:
geography
East Midlands           413
East of England         413
London                  413
North East        

In [173]:
ecoli = pd.read_csv("/Users/admin/AMR/data/cleaned/ecoli_combined_cleaned.csv")
len(ecoli)

3717

In [139]:
keep_columns = [
    "topic",
    "geography",
    "stratum",
    "date",
    "metric_value",
    "in_reporting_delay_period"
]

dataframes = []

In [140]:
print("Files being loaded:")
for csv_file in file_dir.glob("*.csv"):
    print(csv_file.name)


Files being loaded:
strep.csv
p bac.csv
ecoli.csv
efaecalis_efaecium_combined.csv
klebsiella.csv


In [194]:
def standardise_amr_df(df, organism_name):
    """
    Standardises a raw AMR dataframe into modelling-ready format.
    Returns: geography | stratum | date | resistance_pct | organism
    """
    
    # --- Resistance extraction ---
    if 'metric_value' in df.columns:
        resistance = pd.to_numeric(df['metric_value'], errors='coerce')
    elif 'metric value' in df.columns:
        resistance = pd.to_numeric(df['metric value'], errors='coerce')
    else:
        raise ValueError("No metric column found")

    # --- Date ---
    date = pd.to_datetime(df['date'], dayfirst=False, errors='coerce')

    clean = pd.DataFrame({
        'geography': df['geography'],
        'stratum': df['stratum'],
        'date': date,
        'resistance_pct': resistance,
        'organism': organism_name
    })

    clean = clean.dropna(subset=['date', 'resistance_pct'])

    return clean


In [159]:
efaec = pd.read_csv("/Users/admin/AMR/data/cleaned/efaecalis_efaecium_combined.csv")

efaecalis = standardise_amr_df(
    efaec[efaec['topic'] == 'E-faecalis'], 
    "e-faecalis"
)

efaecium = standardise_amr_df(
    efaec[efaec['topic'] == 'E-faecium'], 
    "e-faecium"
)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/admin/AMR/data/cleaned/efaecalis_efaecium_combined.csv'

In [160]:
len(efaecium)

531

In [174]:
kleb = pd.read_csv("/Users/admin/AMR/data/raw/data/klebsiella.csv")
klebsiella = standardise_amr_df(
    kleb[kleb['topic']== 'K-pneumoniae'],
    "K-pneumoniae"
)
strep = pd.read_csv("/Users/admin/AMR/data/raw/data/strep.csv")

s_pneumoniae = standardise_amr_df(
    strep[strep['topic'] == 'S-pneumoniae'],
    "S-pneumoniae"
)

pseudo = pd.read_csv("/Users/admin/AMR/data/raw/data/p bac.csv")

p_aeruginosa = standardise_amr_df(
    pseudo[pseudo['topic'] == 'P-aeruginosa'],
    "P-aeruginosa"
)




In [180]:
ecoli = ecoli.rename(columns={'topic': 'organism'})
ecoli['organism'] = 'e_coli'


In [181]:
print(ecoli.columns.tolist())


['geography', 'stratum', 'date', 'resistance_pct', 'organism']


In [182]:
len(klebsiella) 


3717

In [183]:
len(ecoli)

3717

In [184]:
len(s_pneumoniae)

1056

In [185]:
len(p_aeruginosa)

2655

In [186]:
dfs = {
    "ecoli": ecoli,
    "klebsiella": klebsiella,
    "p_aeruginosa": p_aeruginosa,
    "s_pneumoniae": s_pneumoniae,
    "efaecalis": efaecalis,
    "efaecium": efaecium
}

for name, df in dfs.items():
    print(f"\n{name}")
    print(df.columns.tolist())
    print("Rows:", len(df))



ecoli
['geography', 'stratum', 'date', 'resistance_pct', 'organism']
Rows: 3717

klebsiella
['geography', 'stratum', 'date', 'resistance_pct', 'organism']
Rows: 3717

p_aeruginosa
['geography', 'stratum', 'date', 'resistance_pct', 'organism']
Rows: 2655

s_pneumoniae
['geography', 'stratum', 'date', 'resistance_pct', 'organism']
Rows: 1056

efaecalis
['geography', 'stratum', 'date', 'resistance_pct', 'organism']
Rows: 531

efaecium
['geography', 'stratum', 'date', 'resistance_pct', 'organism']
Rows: 531


In [187]:
master = pd.concat(
    [ecoli, klebsiella, p_aeruginosa, s_pneumoniae, efaecalis, efaecium],
    ignore_index=True
)


In [188]:
len(master)

12207

In [189]:
print("Total rows:", len(master))

print("\nRows per organism:")
print(master['organism'].value_counts())


Total rows: 12207

Rows per organism:
organism
e_coli          3717
K-pneumoniae    3717
P-aeruginosa    2655
S-pneumoniae    1056
e-faecalis       531
e-faecium        531
Name: count, dtype: int64


In [190]:
master = master.sort_values(
    ['organism', 'geography', 'stratum', 'date']
).reset_index(drop=True)


In [191]:
output_path = Path("/Users/admin/AMR/data/cleaned/master_cleaned.csv")
master.to_csv(output_path, index=False)

print(f"✅ Master dataset saved: {output_path}")


✅ Master dataset saved: /Users/admin/AMR/data/cleaned/master_cleaned.csv


In [192]:
import pandas as pd


amr_new = pd.read_csv(
    "/Users/admin/AMR/data/cleaned/master_cleaned.csv"
)


In [193]:
len(amr_new)


12207